# Face Detection and Recognition

## Importing necessary libraries

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [2]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

## Extracting faces for training dataset

In [3]:
def face_extractor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    
    return cropped_face

In [8]:
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        face = face_extractor(frame)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        filename = "faces/user"+str(count)+".jpg"
        cv2.imwrite(filename, face)
        count+=1
        cv2.putText(face, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, color = (0,0,255), thickness= 2)
        cv2.imshow("Face", face)
    else:
        print("Face not found")
    if cv2.waitKey(1)==13 or count==200:
        break
cap.release()
cv2.destroyAllWindows()

In [4]:
filepath = ["faces/"+str(i) for i in listdir("faces")]
filepath

['faces/user0.jpg',
 'faces/user1.jpg',
 'faces/user10.jpg',
 'faces/user100.jpg',
 'faces/user101.jpg',
 'faces/user102.jpg',
 'faces/user103.jpg',
 'faces/user104.jpg',
 'faces/user105.jpg',
 'faces/user106.jpg',
 'faces/user107.jpg',
 'faces/user108.jpg',
 'faces/user109.jpg',
 'faces/user11.jpg',
 'faces/user110.jpg',
 'faces/user111.jpg',
 'faces/user112.jpg',
 'faces/user113.jpg',
 'faces/user114.jpg',
 'faces/user115.jpg',
 'faces/user116.jpg',
 'faces/user117.jpg',
 'faces/user118.jpg',
 'faces/user119.jpg',
 'faces/user12.jpg',
 'faces/user120.jpg',
 'faces/user121.jpg',
 'faces/user122.jpg',
 'faces/user123.jpg',
 'faces/user124.jpg',
 'faces/user125.jpg',
 'faces/user126.jpg',
 'faces/user127.jpg',
 'faces/user128.jpg',
 'faces/user129.jpg',
 'faces/user13.jpg',
 'faces/user130.jpg',
 'faces/user131.jpg',
 'faces/user132.jpg',
 'faces/user133.jpg',
 'faces/user134.jpg',
 'faces/user135.jpg',
 'faces/user136.jpg',
 'faces/user137.jpg',
 'faces/user138.jpg',
 'faces/user139.jp

## Creating Training Data for model

In [5]:
x_train, y_train = [],[]
for i, file in enumerate(filepath):
    image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    x_train.append(np.asarray(image, dtype = np.uint8))
    y_train.append(np.asarray(1, dtype = np.uint8))

y_train = np.array(y_train, dtype = np.int32)   

In [6]:
model = cv2.face.LBPHFaceRecognizer_create()
model.train(x_train, y_train)
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

## Face detection and recognition

In [56]:
def face_detector(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,)
    if faces is ():
        return image, []
    for  (x,y,w,h) in faces:
        cv2.rectangle(image,pt1 = (x,y), pt2 = (x+w, y+h),color = (0,255,255), thickness = 2)
        roi = image[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200,200))
    
    return image, roi    

In [62]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        result = model.predict(gray)
        if result[1]<500:
            confidence = int(100 * (1 - (result[1]) / 300))
        
        cv2.putText(image,("Confidence Level:" + str(confidence)+"%"), (50,50), 
                    cv2.FONT_HERSHEY_COMPLEX, 1, color = (255,0,0), thickness= 2)
        
        
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 55, 255), 2)
            cv2.imshow('FACE DETECTOR', image)
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 55, 0), 2)
            cv2.imshow('FACE DETECTOR', image)
        
    except:
        cv2.putText(image, "Face Not Detected", (100,100), cv2.FONT_HERSHEY_COMPLEX,1,
                   color = (0,255,0), thickness = 2)
        cv2.imshow("FACE DETECTOR", image)
        pass
        
    if cv2.waitKey(1)==13:
        break

cap.release()
cv2.destroyAllWindows()